In [15]:
import unittest
import importlib as i
import cosmology as c
import deprecated as d
import numpy as np
from scipy.special import erf

In [74]:
class TestCosmology(unittest.TestCase):
    
    @unittest.SkipTest
    def test_set_default(self):
        c.cosm_model = [1,2,5,3,6]
        c.H_0 = c.O_M = c.O_L = c.O_B = 69
        self.assertEqual(c.cosm_model, [1,2,5,3,6])
        self.assertEqual(c.O_M, 69)
        self.assertEqual(c.H_0, 69)
        
        c.set_default()
        self.assertEqual(c.cosm_model, [0.286, 0.714, 0.049, 70])
        self.assertEqual(c.O_M, 0.286)
        self.assertEqual(c.O_L, 0.714)
        self.assertEqual(c.O_B, 0.049)
        self.assertEqual(c.H_0, 70*c.kmsMpc_to_s)

    @unittest.SkipTest    
    def test_update_cmodel(self):
        new_model = [0.284, 0.755, 0.010, 50]
        c.update_cmodel(new_model)
        self.assertEqual(c.cosm_model, [0.284, 0.755, 0.010, 50])
        self.assertEqual(c.H_0, 50*c.kmsMpc_to_s)
        self.assertEqual(c.O_M, 0.284)
        self.assertEqual(c.O_L, 0.755)
        self.assertEqual(c.O_B, 0.010)
        
        new_model.append(69)
        self.assertEqual(c.cosm_model, [0.284, 0.755, 0.010, 50])
        
        c.set_default()
    
    @unittest.SkipTest
    def test_sort_by_first(self):
        l1, l2 = c.sort_by_first([4,3,5,-1,2], [5,69,8,72,-11])
        self.assertEqual(l1.tolist(), [-1,2,3,4,5])
        self.assertEqual(l2.tolist(), [72,-11,69,5,8])
        
        l1, l2 = c.sort_by_first([5,4,3,2], [-5,-6,-7,8])
        self.assertEqual(l1.tolist(), [2,3,4,5])
        self.assertEqual(l2.tolist(), [8,-7,-6,-5])
        
        #check if sort in place
        l1, l2 = c.sort_by_first([9,9,3,9,9], [5,-2,3,-4,99])
        self.assertEqual(l1.tolist(), [3,9,9,9,9])
        self.assertEqual(l2.tolist(), [3,5,-2,-4,99])
        
        #check if it works for numpy arrays as well
        l1, l2 = c.sort_by_first(np.array([4,3,5,-1,2]), np.array([5,69,8,72,-11]))
        self.assertEqual(l1.tolist(), [-1,2,3,4,5])
        self.assertEqual(l2.tolist(), [72,-11,69,5,8])
        
        l1, l2 = c.sort_by_first(np.array([5,4,3,2]), np.array([-5,-6,-7,8]))
        self.assertEqual(l1.tolist(), [2,3,4,5])
        self.assertEqual(l2.tolist(), [8,-7,-6,-5])
        
        l1, l2 = c.sort_by_first(np.array([9,9,3,9,9]), np.array([5,-2,3,-4,99]))
        self.assertEqual(l1.tolist(), [3,9,9,9,9])
        self.assertEqual(l2.tolist(), [3,5,-2,-4,99])
    
    @unittest.SkipTest
    def test_integrate_mult_ends(self):
        cubed = lambda x: x**3
        int_cubed = lambda x: x**4/4
        expsq = lambda x: np.exp(-x**2)
        int_expsq = lambda x: (np.pi)**0.5/2*erf(x)
        start = 2.3
        end1 = 7.4
        end2 = np.array([7.4, 3.74, 8.123, 3, 6.68, 99])
        
        self.assertAlmostEqual(c.integrate_mult_ends(cubed, start, end1), int_cubed(end1)-int_cubed(start), delta=1e-8)
        np.testing.assert_almost_equal(c.integrate_mult_ends(cubed, start, end2), int_cubed(end2)-int_cubed(start), 8)
        
        self.assertAlmostEqual(c.integrate_mult_ends(expsq, start, end1), int_expsq(end1)-int_expsq(start), delta=1e-8)
        np.testing.assert_almost_equal(c.integrate_mult_ends(expsq, start, end2), int_expsq(end2)-int_expsq(start), 8)
        
    @unittest.SkipTest
    def test_E(self):
        self.assertEqual(c.E(0), (c.O_M + c.O_L)**0.5)
        self.assertEqual(c.E(1), (8.0*c.O_M + c.O_L)**0.5)
        self.assertEqual(c.E(0.5), (3.375*c.O_M + c.O_L)**0.5)
        self.assertEqual(c.E(55.19482), ((56.19482)**3*c.O_M + c.O_L)**0.5)
        
        ans = np.array([(c.O_M + c.O_L)**0.5, (8.0*c.O_M + c.O_L)**0.5, (3.375*c.O_M + c.O_L)**0.5, ((56.19482)**3*c.O_M + c.O_L)**0.5])
        self.assertEqual(c.E(np.array([0,1,0.5,55.19482])).tolist(), ans.tolist())
    
    @unittest.SkipTest
    def test_D_C(self):
        self.assertAlmostEqual(c.D_C(2.12384), d.D_C_old(2.12384), delta=1e-7)
        self.assertAlmostEqual(c.D_C(2.12384), 1.26580797970764, delta=1e-11) # value calcualted through integral-calculator
        self.assertAlmostEqual(c.D_C(55.123854), 2.870559850206997, delta=1e-11) # value calcualted through integral-calculator
        
        input = np.array([50,1,3,25,4])
        np.testing.assert_almost_equal(c.D_C(input), d.D_C_old(input), 7)
        np.testing.assert_almost_equal(c.D_C(input), np.array([2.846083449712865, 0.7779423776121797, 1.504993571457544, 2.636332485477817, 1.699640679205983]), 11)

    @unittest.SkipTest
    def test_D_L(self):
        self.assertAlmostEqual(c.D_L(2.12384), 3.12384*1.26580797970764, delta=1e-11)
        self.assertAlmostEqual(c.D_L(55.123854), 56.123854*2.870559850206997, delta=1e-11) # value calcualted through integral-calculator
        
        exp = np.array([2.846083449712865, 0.7779423776121797, 1.504993571457544, 2.636332485477817, 1.699640679205983]) * np.array([51,2,4,26,5])
        np.testing.assert_almost_equal(c.D_L(np.array([50,1,3,25,4])), exp, 11)
    
    @unittest.SkipTest
    def test_dDMdz_Arcus(self):
        self.assertAlmostEqual(c.dDMdz_Arcus(0), 1010.44805898, delta=1e-8)
        self.assertAlmostEqual(c.dDMdz_Arcus(1.5), 1109.61896919, delta=1e-8)
        self.assertAlmostEqual(c.dDMdz_Arcus(2.293), 1006.60923609, delta=1e-8)
        self.assertAlmostEqual(c.dDMdz_Arcus(4.87), 664.356689675, delta=1e-8)
        self.assertAlmostEqual(c.dDMdz_Arcus(6.11991), 604.853405924, delta=1e-8)
        self.assertAlmostEqual(c.dDMdz_Arcus(9.326), 0, delta=1e-8)
        
        input = np.array([6.11991, 4.87, 0, 1.5, 9.326, 2.293])
        exp = np.array([604.853405924, 664.356689675, 1010.44805898, 1109.61896919, 0, 1006.60923609])
        np.testing.assert_almost_equal(c.dDMdz_Arcus(input), exp, 7)
    
    @unittest.SkipTest
    def test_dDMdz_Zhang(self):
        self.assertAlmostEqual(c.dDMdz_Zhang(0), 848.776369544, delta=1e-8)
        self.assertAlmostEqual(c.dDMdz_Zhang(1.5), 932.07993412, delta=1e-8)
        self.assertAlmostEqual(c.dDMdz_Zhang(2.293), 845.551758312, delta=1e-8)
        self.assertAlmostEqual(c.dDMdz_Zhang(4), 702.798992847, delta=1e-8)
        self.assertAlmostEqual(c.dDMdz_Zhang(6.11991), 592.756337805, delta=1e-8)
        self.assertAlmostEqual(c.dDMdz_Zhang(9.326), 493.346886956, delta=1e-8)
        
        input = np.array([6.11991, 4, 0, 1.5, 9.326, 2.293])
        exp = np.array([592.756337805, 702.798992847, 848.776369544, 932.07993412, 493.346886956, 845.551758312])
        np.testing.assert_almost_equal(c.dDMdz_Zhang(input), exp, 7)
    
    @unittest.SkipTest
    def test_DM(self):
        self.assertAlmostEqual(c.DM(1.75432), d.DM_V1(1.75432), delta=1)
        self.assertAlmostEqual(c.DM(1.75432, dDMdz=c.dDMdz_Zhang), d.DM_V1(1.75432, dDMdz=c.dDMdz_Zhang), delta=1)
        input = np.array([9, 1.7, 3.1257, 0.135, 0.743, 5.812, 7.952785])
        np.testing.assert_almost_equal(c.DM(input), d.DM_V1(input), 0)
        np.testing.assert_almost_equal(c.DM(input, dDMdz=c.dDMdz_Zhang), d.DM_V1(input, dDMdz=c.dDMdz_Zhang), 0)
    
    def test_z_DM(self):
        DM1 = 719.27384
        self.assertAlmostEqual(c.DM(c.z_DM(DM1)), DM1, delta = 1e-7)
        self.assertAlmostEqual(c.DM(c.z_DM(DM1, "Zhang"), dDMdz=c.dDMdz_Zhang), DM1, delta = 1e-5)
        
        z1 = 0.517394
        self.assertAlmostEqual(c.z_DM(c.DM(z1)), z1, delta = 1e-9)
        self.assertAlmostEqual(c.z_DM(c.DM(z1, dDMdz=c.dDMdz_Zhang), "Zhang"), z1, delta = 1e-9)
        
        DM2 = np.array([719.27384, 318.9591, 951.7174, 1502.58448, 6012.2384, 4815.234])
        np.testing.assert_almost_equal(c.DM(c.z_DM(DM2)), DM2, 3)
        
        #TODO: #1 continue testing z_DM, and the rest of the cosmology functions

In [75]:
i.reload(c)
#i.reload(d)
unittest.main(argv=[''], verbosity=2, exit=False)

test_z_DM (__main__.TestCosmology) ... ok

----------------------------------------------------------------------
Ran 1 test in 1.070s

OK
